# Hierarchical clustering

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # Plot library
from scipy.cluster.hierarchy import dendrogram, linkage # Clustering library
import gc

# some setting for this notebook to actually show the graphs inline
# you probably won't need this
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation

from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist


In [2]:
df_songs = pd.read_csv('../Data/songs.csv')
print(df_songs.head())

                                        song_id  song_length genre_ids  \
0  CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=       247640       465   
1  o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=       197328       444   
2  DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=       231781       465   
3  dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=       273554       465   
4  W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=       140329       726   

        artist_name                            composer     lyricist  language  
0  張信哲 (Jeff Chang)                                  董貞          何啟弘       3.0  
1         BLACKPINK  TEDDY|  FUTURE BOUNCE|  Bekuh BOOM        TEDDY      31.0  
2      SUPER JUNIOR                                 NaN          NaN      31.0  
3             S.H.E                                 湯小康          徐世珍       3.0  
4              貴族精選                         Traditional  Traditional      52.0  


In [3]:
df_songs['artist_name'] = df_songs['artist_name'].astype('category')
foo = lambda x: pd.Series([i for i in reversed(x.split('|'))])
rev = df_songs['artist_name'].apply(foo)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [ ]:
categories = ['genre_ids', 'artist_name', 'language']

# for col in categories:
#     print(len(df_songs[col].value_counts()))
#     print(df_songs[col].value_counts())



threshold = 100
for col in categories:
    value_counts = df_songs.stack().value_counts() # Entire DataFrame 
    to_remove = value_counts[value_counts <= threshold].index
    df_songs[col].replace(to_remove, np.nan, inplace=True)

for col in categories:
    print(col, len(df_songs[col].value_counts()))
#     print(df_songs[col].value_counts())

In [10]:
Dummies_songs = pd.get_dummies(data=df_songs, prefix_sep='|',columns=categories)
print(Dummies_songs.head())

                                        song_id  song_length  \
0  CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=       247640   
1  o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=       197328   
2  DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=       231781   
3  dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=       273554   
4  W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=       140329   

        artist_name                            composer     lyricist  \
0  張信哲 (Jeff Chang)                                  董貞          何啟弘   
1         BLACKPINK  TEDDY|  FUTURE BOUNCE|  Bekuh BOOM        TEDDY   
2      SUPER JUNIOR                                 NaN          NaN   
3             S.H.E                                 湯小康          徐世珍   
4              貴族精選                         Traditional  Traditional   

   genre_ids|1000  genre_ids|1000|2154|751  genre_ids|1007  genre_ids|1011  \
0               0                        0               0               0   
1               0         

In [ ]:
Z = linkage(Dummies_songs, 'ward')